In [22]:
!pip install torch transformers coremltools

In [23]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

# Create dummy input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

In [26]:
# Trace the model
model = BertModel.from_pretrained("google-bert/bert-base-uncased", torchscript=True)
model.eval()
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5055: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [27]:
import coremltools as ct

# Convert into Core ML-compatible format
coreml_model = ct.convert(
  traced_model,
    inputs=[
      ct.TensorType(shape=dummy_input[0].shape, name="input_ids"),  # Renamed and matched to the first input
      ct.TensorType(shape=dummy_input[1].shape, name="attention_mask")  # Added the second input
  ],
  outputs=[
      ct.TensorType(name="last_hidden_state"),  # Output for the last hidden state
      ct.TensorType(name="pooler_output")  # Output for the pooler output
  ])

Running MIL default pipeline:  65%|██████▌   | 58/89 [00:01<00:01, 26.29 passes/s]/usr/local/lib/python3.10/dist-packages/coremltools/converters/mil/mil/ops/defs/iOS15/elementwise_unary.py:889: RuntimeWarning: overflow encountered in cast
  return input_var.val.astype(dtype=string_to_nptype(dtype_val))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 51.50 passes/s]


In [28]:
# Save the Core ML model
coreml_model.save("bert-base-uncased.mlpackage")

In [29]:
!zip -r bert-base-uncased.mlpackage.zip /content/bert-base-uncased.mlpackage

updating: content/bert-base-uncased.mlpackage/ (stored 0%)
updating: content/bert-base-uncased.mlpackage/Manifest.json (deflated 59%)
updating: content/bert-base-uncased.mlpackage/Data/ (stored 0%)
updating: content/bert-base-uncased.mlpackage/Data/com.apple.CoreML/ (stored 0%)
updating: content/bert-base-uncased.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
updating: content/bert-base-uncased.mlpackage/Data/com.apple.CoreML/weights/weight.bin (deflated 8%)
updating: content/bert-base-uncased.mlpackage/Data/com.apple.CoreML/model.mlmodel (deflated 89%)


In [21]:
from google.colab import files
files.download("bert-base-uncased.mlpackage.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>